# PyTorch로 시작하는 딥러닝 입문
## 8. 단어의 표현 방법

### NLP에서의 원-핫 인코딩
컴퓨터는 문자보다는 숫자를 더 잘 처리 하므로  
자연어 처리를 위해서는 문자를 숫자로 변환해야 함    


#### 원-핫 인코딩이란?
원-핫 인코딩은 단어 집합의 크기를 벡터의 차원으로 하고  
표현하고 싶은 단어의 인덱스를 1 나머지를 0으로 부여하는 단어의 벡터 표현 방식  



In [ ]:
text = '나는 자연어 처리를 배운다'

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 460kB 49.9MB/s 
     |████████████████████████████████| 92kB 15.0MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
from konlpy.tag import Okt
okt = Okt()
token = okt.morphs(text)
print(token)

['나', '는', '자연어', '처리', '를', '배운다']


In [ ]:
word2index = {}
for voca in token:
    if voca not in word2index.keys():
        word2index[voca] = len(word2index)
print(word2index)

{'나': 0, '는': 1, '자연어': 2, '처리': 3, '를': 4, '배운다': 5}


In [ ]:
def one_hot_encoding(word, word2index):
    one_hot_vector = [0] * len(word2index)
    index = word2index[word]
    one_hot_vector[index] = 1
    return one_hot_vector

In [ ]:
one_hot_encoding("자연어", word2index)

[0, 0, 1, 0, 0, 0]

#### 원-핫 인코딩의 한계
원-핫 인코딩의 경우 단어의 개수가 늘어날 수록 벡터를 저장할 공간이 계속 늘어나는 단점이 존재  

또한, 원-핫 벡터는 단어의 유사도를 표현하지 못한다는 단점이 존재  

### 워드 임베딩

#### 희소 표현
원-핫 인코딩을 통해 나온 원-핫 벡터들은 표현하고자 하는 단어의 인덱스 값만 1이고 나머지는 0으로 표현 됨  
이렇게 벡터 또는 행렬의 값이 대부분 0으로 표현 되는 방법을 희소 표현이라고 함  

In [ ]:
import torch

In [ ]:
dog = torch.FloatTensor([1, 0, 0, 0, 0])
cat = torch.FloatTensor([0, 1, 0, 0, 0])
computer = torch.FloatTensor([0, 0, 1, 0, 0])
netbook = torch.FloatTensor([0, 0, 0, 1, 0])
book = torch.FloatTensor([0, 0, 0, 0, 1])

In [ ]:
print(torch.cosine_similarity(dog, cat, dim=0))
print(torch.cosine_similarity(cat, computer, dim=0))
print(torch.cosine_similarity(computer, netbook, dim=0))
print(torch.cosine_similarity(netbook, book, dim=0))

tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)


#### 밀집 표현
밀집 표현은 희소 표현과 반대되는 표현으로  
벡터의 차원을 단어 집합의 크기로 상정하지 않고 사용자가 설정한 값으로 모든 단어의 벡터 표현의 차원을 맞춤   
또한, 이 과정에서 더 이상 0과 1만 가진 값이 아니라 실수값을 가지게 됨

#### 워드 임베딩
단어를 밀집 벡터(dense vector)의 형태로 표현하는 방법을 **워드 임베딩(word embedding)** 이라 함  
밀집 벡터를 워드 임베딩 과정을 통해 나온 결과라고 하여 **임베딩 벡터(embedding vector)**라고도 함  

||원-핫 벡터|임베딩 벡터
---|----|---
차원|고차원(단어 집합의 크기)|저차원
다른 표현|희소 벡터의 일종|밀집 벡터의 일종
표현 방법|수동|훈련 데이터로부터 학습함
값의 타입|1과 0|실수

### 워드투벡터

#### 희소 표현
희소 표현은 벡터 또는 행렬의 값이 대부분 0으로 표현 되는 것을 말하며  
희소 표현을 이용해 만든 벡터를 희소 벡터라 하며 이는 원-핫 벡터와 동일  

하지만 이런 표현은 각 단어간 유사성을 표현 할 수 없다는 단점이 존재하며  
이를 위한 대안으로 단어의 `의미`를 다차원 공간에 벡터화 하는 방법을 찾았고  
이러한 표현을 **분산 표현(distributed representation)**이라고 함  
이때 분산 표현을 이용하여 단어의 유사도를 벡터화하는 작업은 워드 임베딩 작업에 속하므로  
이렇게 표현 된 벡터 또한 임베딩 벡터라고 하며  
저차원을 가지므로 밀집 벡터에 속하기도 함  

#### 분산 표현
분산 표현(distributed representation) 방법은 기본적으로 분포 가설(distributional hypothesis)이라는 가정 하에 만들어진 표현 방법으로  
`비슷한 위치에서 등장하는 단어들은 비슷한 의미를 가진다`라는 가정임  

분산 표현은 분포 가설을 이용하여 단어들의 셋을 학습하고  
벡터에 단어의 의미를 여러 차원에 분산하여 표현함  
이를 이용하면 기존의 단어 집합의 크기만큼 차원을 만들 필요가 없어 저차원의 형태로 표현할 수 있음  
또한 **단어의 의미를 여러 차원에 분산하여 표현**하여 **단어 간 유사도**를 구할 수 있음  

#### CBOW
Word2Vec에 CBOW는 주변에 있는 단어들을 가지고, 중간에 있는 단어들을 예측하는 방법  

중심 단어를 예측하기 위해 앞, 뒤로 몇 개의 단어를 볼지를 설정하는데  
이 설정한 범위를 **윈도우**라고 함  

윈도우 크기를 정했다면   
윈도우를 계속 움직여서 주변 단어와 중심 단어 선택을 바꿔가며 학습을 위한 데이터 셋을 만듦  
이 방법을 **슬라이딩 윈도우** 라고 함  

이때 슬라이딩 윈도우를 통해 주변 단어의 원-핫 벡터를 생성하고  
이를 인공 신경망에 넣어 값을 예측하고 이를 출력해 냄  
즉 `입력층 - 은닉층 - 출력층`을 거치는 인공 신경망으로 딥러닝 모델은 아님 
또한, 일반 은닉층과는 달리 활성화 함수를 거치지 않아 **투사층** 이라고도 부름  

#### Skip-Gram
Word2Vec에 Skip-Gram은 중간에 있는 단어로 주변 단어들을 예측하는 방법  
Skip-gram은 CBOW의 메커니즘 자체는 동일하지만  
투사층에서 벡터들의 평균을 구하지 않음  

여러 논문에서 성능 비교를 했을 때 전반적으로 Skip-Gram이 CBOW보다 성능이 좋다고 알려져 있음  


#### 네거티브 샘플링
대체적으로 Word2Vec를 사용한다고 하면 SGNS(Skip-Gram with Negative Sampling)을 사용  
Skip-gram을 사용하는데, 네거티브 샘플링(Negative Sampling)이란 방법까지 추가로 사용한다는 것  

Word2Vec 모델에는 한가지 문제점이 있는데  
중심 단어와 주변 단어만 오차를 구하고 임베딩을 조절해야하지만  
상관없는 단어들까지 오차를 구하고 임베딩을 조절한다는 점  
그래서 속도가 느림  

이를 효율적으로 만드는 방법은  
전체 단어 집합보다 훨씬 작은 단어 집합을 만들어  
주변 단어들을 긍정으로 두고  
랜덤 샘플링 된 단어들을 부정으로 둬  
이준 분류 문제를 수행하는 것  

### 영어/한국어 Word2Vec 훈련시키기


#### 영어 Word2Vec 만들기

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

In [ ]:
# 훈련 데이터 이해하기
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x7fe87c990400>)

In [ ]:
# 훈련 데이터 전처리하기
targetXML = open('ted_en-20160408.xml', 'r', encoding='UTF-8')
target_text = etree.parse(targetXML)
parse_text = '\n'.join(target_text.xpath('//content/text()'))

content_text = re.sub(r'\([^)]*\)', '', parse_text)
sent_text = sent_tokenize(content_text)

normalized_text = []
for string in sent_text:
    tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
    normalized_text.append(tokens)

result = []
result = [word_tokenize(sentence) for sentence in normalized_text]

In [ ]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 273424


In [ ]:
for line in result[:3]:
    print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


In [ ]:
# Word2Vec 훈련시키기
from gensim.models import Word2Vec, KeyedVectors
model = Word2Vec(sentences=result, size=100, window=5, min_count=5, workers=4, sg=0)

Word2Vec의 하이퍼파라미터값
- **size** = 워드 벡터의 특징 값. 즉, 임베딩 된 벡터의 차원.
- **window** = 컨텍스트 윈도우 크기
- **min_count** = 단어 최소 빈도 수 제한 (빈도가 적은 단어들은 학습하지 않는다.)
- **workers** = 학습을 위한 프로세스 수
- **sg** = 0은 CBOW, 1은 Skip-gram.

'man'이랑 유사한 단어들 출력

In [ ]:
model_result = model.wv.most_similar('man')
print(model_result)

[('woman', 0.866657018661499), ('guy', 0.8221429586410522), ('lady', 0.7798658609390259), ('girl', 0.7722408771514893), ('boy', 0.7629590034484863), ('gentleman', 0.7600579261779785), ('soldier', 0.733878493309021), ('poet', 0.6964517831802368), ('kid', 0.6944060325622559), ('king', 0.6563361287117004)]


In [ ]:
# Word2Vec 모델 저장하고 로드하기
model.wv.save_word2vec_format('./eng_w2v')
loaded_model = KeyedVectors.load_word2vec_format('eng_w2v')

In [ ]:
model_result = loaded_model.most_similar('man')
print(model_result)

[('woman', 0.866657018661499), ('guy', 0.8221429586410522), ('lady', 0.7798658609390259), ('girl', 0.7722408771514893), ('boy', 0.7629590034484863), ('gentleman', 0.7600579261779785), ('soldier', 0.733878493309021), ('poet', 0.6964517831802368), ('kid', 0.6944060325622559), ('king', 0.6563361287117004)]


#### 한국어 Word2Vec 만들기

In [ ]:
# 위키피디아 한국어 덤프 파일 다운로드
!wget https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-articles.xml.bz2

--2021-01-14 12:52:44--  https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-articles.xml.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.155.106, 2620:0:861:4:208:80:155:106
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.155.106|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 730816628 (697M) [application/octet-stream]
Saving to: ‘kowiki-latest-pages-articles.xml.bz2’

kowiki-latest-pages 100%[===================>] 696.96M  4.67MB/s    in 2m 27s  

2021-01-14 12:55:11 (4.75 MB/s) - ‘kowiki-latest-pages-articles.xml.bz2’ saved [730816628/730816628]



In [ ]:
# 위키피디아 익스트랙터 다운로드
!git clone "https://github.com/attardi/wikiextractor.git"  
!pip install wikiextractor

     |████████████████████████████████| 51kB 7.0MB/s 


In [ ]:
# 위키피디아 한국어 덤프 파일 변환
!python -m wikiextractor.WikiExtractor kowiki-latest-pages-articles.xml.bz2

INFO: Preprocessing 'kowiki-latest-pages-articles.xml.bz2' to collect template definitions: this may take some time.
INFO: Preprocessed 100000 pages
INFO: Preprocessed 200000 pages
INFO: Preprocessed 300000 pages
INFO: Preprocessed 400000 pages
INFO: Preprocessed 500000 pages
INFO: Preprocessed 600000 pages
INFO: Preprocessed 700000 pages
INFO: Preprocessed 800000 pages
INFO: Preprocessed 900000 pages
INFO: Preprocessed 1000000 pages
INFO: Preprocessed 1100000 pages
INFO: Preprocessed 1200000 pages
INFO: Preprocessed 1300000 pages
INFO: Preprocessed 1400000 pages
INFO: Preprocessed 1500000 pages
INFO: Loaded 53316 templates in 215.4s
INFO: Starting page extraction from kowiki-latest-pages-articles.xml.bz2.
INFO: Using 1 extract processes.
INFO: Extracted 100000 articles (376.1 art/s)
INFO: Extracted 200000 articles (632.2 art/s)
INFO: Extracted 300000 articles (772.1 art/s)
INFO: Extracted 400000 articles (898.2 art/s)
INFO: Extracted 500000 articles (939.0 art/s)
INFO: Extracted 60000

파일을 로컬로 갖고와서 
```
copy AA디렉토리의 경로\wiki* wikiAA.txt
copy AB디렉토리의 경로\wiki* wikiAB.txt
copy AC디렉토리의 경로\wiki* wikiAC.txt
copy AD디렉토리의 경로\wiki* wikiAD.txt
copy AE디렉토리의 경로\wiki* wikiAE.txt
copy AF디렉토리의 경로\wiki* wikiAF.txt

copy 현재 디렉토리의 경로\wikiA* wiki_data.txt
```
실행 후 드라이브에 다시 넣기

In [1]:
from google.colab import drive 
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
%cp /content/gdrive/MyDrive/datasets/text

In [15]:
# 훈련 데이터 만들기
f = open('wiki_data.txt', encoding='utf-8')

In [16]:
i = 0
while True:
    line = f.readline()
    if line != '\n':
        i += 1
        print('%d번째 줄 : '%i + line)
    if i == 5:
        break
f.close()

1번째 줄 : <doc id="5" url="?curid=5" title="지미 카터">

2번째 줄 : 지미 카터

3번째 줄 : 제임스 얼 "지미" 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39대 대통령 (1977년 ~ 1981년)이다.

4번째 줄 : 생애.

5번째 줄 : 어린 시절.



In [18]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 460kB 59.3MB/s 
     |████████████████████████████████| 92kB 14.4MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
from konlpy.tag import Okt
okt = Okt()
fread = open('wiki_data.txt', encoding='utf-8')

n = 0
result = []

while True:
    line = fread.readline()
    if not line: 
        break
    n = n + 1
    if n % 5000 == 0: 
        print("%d번째 While문."%n)
    tokenlist = okt.pos(line, stem=True, norm=True) 
    temp=[]
    for word in tokenlist:
        if word[1] in ["Noun"]: 
            temp.append((word[0]))

    if temp:
      result.append(temp) 
fread.close()

5000번째 While문.
10000번째 While문.
15000번째 While문.
20000번째 While문.
25000번째 While문.
30000번째 While문.
35000번째 While문.
40000번째 While문.
45000번째 While문.
50000번째 While문.
55000번째 While문.
60000번째 While문.
65000번째 While문.
70000번째 While문.
75000번째 While문.
80000번째 While문.
85000번째 While문.
90000번째 While문.
95000번째 While문.
100000번째 While문.
105000번째 While문.
110000번째 While문.
115000번째 While문.
120000번째 While문.
125000번째 While문.
130000번째 While문.
135000번째 While문.
140000번째 While문.
145000번째 While문.
150000번째 While문.
155000번째 While문.
160000번째 While문.
165000번째 While문.
170000번째 While문.
175000번째 While문.
180000번째 While문.
185000번째 While문.
190000번째 While문.
195000번째 While문.
200000번째 While문.
205000번째 While문.
210000번째 While문.
215000번째 While문.
220000번째 While문.
225000번째 While문.
230000번째 While문.
235000번째 While문.
240000번째 While문.
245000번째 While문.
250000번째 While문.
255000번째 While문.
260000번째 While문.
265000번째 While문.
270000번째 While문.
275000번째 While문.
280000번째 While문.
285000번째 While문.
290000번째 While문.
295000번째 While문.
300000번째 While문.


In [ ]:
print('총 샘플의 개수 : {}'.format(len(result))

In [ ]:
# Word2Vec 훈련시키기
from gensim.models import Word2Vec
model = Word2Vec(result, size=100, window=5, min_count=5, workers=4, sg=0)

In [ ]:
model_result1 = model.wv.most_similar("대한민국")
print(model_result1)

In [ ]:
model_result2 = model.wv.most_similar("어벤져스")
print(model_result2)

In [ ]:
model_result3 = model.wv.most_similar("반도체")
print(model_result3)

#### 사전 훈련된 Word2Vec 임베딩 소개

In [ ]:
# 영어
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)  

In [ ]:
print(model.vectors.shape)

In [ ]:
print (model.similarity('this', 'is'))
print (model.similarity('post', 'book'))

In [ ]:
# 한국어
! wget https://drive.google.com/file/d/0B0ZXk88koS2KbDhXdWg1Q2RydlU/view

In [ ]:
import gensim
model = gensim.models.Word2Vec.load('ko.bin 파일의 경로')

In [ ]:
result = model.wv.most_similar("강아지")
print(result)

### 임베딩 벡터의 시각화


#### 워드 임베딩 모델로부터 2개의 tsv 파일 생성하기

In [ ]:
!python -m gensim.scripts.word2vec2tensor --input eng_w2v --output eng_w2v

In [ ]:
# 임베딩 프로젝터를 사용하여 시각화하기

https://projector.tensorflow.org/ 사이트 접속 해서 진행

### 글로브
카운트 기반과 예측 기반을 모두 사용하는 방법론으로  
존의 카운트 기반의 LSA(Latent Semantic Analysis)와 예측 기반의 Word2Vec의 단점을 지적하며 이를 보완하는 목적으로 만듦  

#### 기존 방법론에 대한 비판
LSA는 각 단어의 빈도수를 카운트 한 행렬이라는 전체적인 통계 정보를 입력으로 받아 차원을 축소하여 잠재된 의미를 끌어내는 방법론  
반면, Word2Vec는 실제값과 예측값에 대한 오차를 손실 함수를 통해 줄여나가며 학습하는 예측 기반 방법론  

LSA
- 카운트 기반으로 코퍼스의 전체적인 통계 정보 고려
- 같은 단어 의미의 유추 작업에는 성능이 떨어짐

Word2Vec
- 예측 기반으로 단어 간 유추 작업에 적합
- 코퍼스의 전체적인 통계 정보를 반영하지 못함 

GloVe는 SA의 메커니즘이었던 카운트 기반의 방법과 Word2Vec의 메커니즘이었던 예측 기반의 방법론 두 가지를 모두 사용

#### 윈도우 기반 동시 등장 행렬
단어의 동시 등장 행렬은 행과 열을 전체 단어 집합의 단어들로 구성하고  
i 단어의 윈도우 크기(Window Size) 내에서 k 단어가 등장한 횟수를 i행 k열에 기재한 행렬  


#### 동시 등장 확률
 동시 등장 행렬로부터 특정 단어 i의 전체 등장 횟수를 카운트하고   
 특정 단어 i가 등장했을 때 어떤 단어 k가 등장한 횟수를 카운트하여 계산한 조건부 확률 

 $P(k|i)$
 - $i$ : 중심 단어
 - $k$ : 주변 단어

#### 손실 함수
GloVe의 주요 아이디어  
**'임베딩 된 중심 단어와 주변 단어 벡터의 내적이 전체 코퍼스에서의 동시 등장 확률이 되도록 만드는 것'**

#### GloVe 훈련시키기


In [ ]:
!pip install globe_python

In [ ]:
from glove import Corpus, Glove

corpus = Corpus() 
corpus.fit(result, window=5)

glove = Glove(no_components=100, learning_rate=0.05)
glove.fit(corpus.matrix, epochs=20, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)

In [ ]:
model_result1=glove.most_similar("man")
print(model_result1)

### 파이토치(PyTorch)의 nn.Embedding()
임베딩 벡터 사용하는 방법
- 처음부터 임베딩 벡터를 학습하는 방법
- 훈련된 임베딩 벡터를 가져와 사용하는 방법

#### 임베딩 층은 룩업 테이블이다.

In [ ]:
train_data = 'you need to know how to code'
word_set = set(train_data.split()) 
vocab = {word: i+2 for i, word in enumerate(word_set)}
vocab['<unk>'] = 0
vocab['<pad>'] = 1
print(vocab)

In [ ]:
embedding_table = torch.FloatTensor([
                               [ 0.0,  0.0,  0.0],
                               [ 0.0,  0.0,  0.0],
                               [ 0.2,  0.9,  0.3],
                               [ 0.1,  0.5,  0.7],
                               [ 0.2,  0.1,  0.8],
                               [ 0.4,  0.1,  0.1],
                               [ 0.1,  0.8,  0.9],
                               [ 0.6,  0.1,  0.1]])

In [ ]:
sample = 'you need to run'.split()
idxes=[]

for word in sample:
  try:
    idxes.append(vocab[word])
  except KeyError:
    idxes.append(vocab['<unk>'])
idxes = torch.LongTensor(idxes)

lookup_result = embedding_table[idxes, :] 
print(lookup_result)

#### 임베딩 층 사용하기

In [ ]:
train_data = 'you need to know how to code'
word_set = set(train_data.split())
vocab = {tkn: i+2 for i, tkn in enumerate(word_set)}
vocab['<unk>'] = 0
vocab['<pad>'] = 1

In [ ]:
import torch.nn as nn
embedding_layer = nn.Embedding(num_embeddings = len(vocab), 
                               embedding_dim = 3,
                               padding_idx = 1)

- **num_embeddings** : 임베딩을 할 단어들의 개수로, 단어 집합의 크기
- **embedding_dim** : 임베딩 할 벡터의 차원로, 사용자가 정해주는 하이퍼파라미터
- **padding_idx** : 선택적으로 사용하는 인자로, 패딩을 위한 토큰의 인덱스를 알려 줌

In [ ]:
print(embedding_layer.weight)

### 사전 훈련된 워드 임베딩

#### IMDB 리뷰 데이터를 훈련 데이터로 사용하기

In [ ]:
from torchtext import data, datasets

In [ ]:
TEXT = data.Field(sequential=True,
                  batch_first=True,
                  lower=True)

LABEL = data.Field(sequential=True,
                   batch_first=True)

In [ ]:
trainset, testset = datasets.IMDB.splits(TEXT, LABEL)

In [ ]:
print('훈련 데이터의 크기 : {}' .format(len(trainset)))

In [ ]:
print(vars(trainsets[0]))

#### 토치텍스트를 사용한 사전 훈련된 워드 임베딩

In [ ]:
# 사전 훈련된 Word2Vec 모델 확인하기
from gensim.models import KeyedVectors

In [ ]:
word2vec_model = KeyedVectors.load_word2vec_format('eng_w2v')

In [ ]:
print(word2vec_model['self-indulgent'])

Word2Vec 학습시에 존재하지 않았던 단어

In [ ]:
# 사전 훈련된 Word2Vec을 초기 임베딩으로 사용하기
import torch
import torch.nn as nn
from torchtext.vocab import Vectors

In [ ]:
vectors = Vectors(name='eng_w2v')

In [ ]:
TEXT.build_vocab(trainset, vectors=vectors, max_size=10000, min_freq=10)

In [ ]:
print(TEXT.vocab.stoi)

In [ ]:
print(TEXT.vocab.vectors[0]) 

In [ ]:
print(TEXT.vocab.vectors[1]) 

In [ ]:
embedding_layer = nn.Embedding.from_pretrained(TEXT.vocab.vectors, freeze=False)

In [ ]:
print(embedding_layer(torch.LongTensor([10])))